In [1]:
import sys
sys.path.append("../../../../")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pricer.evaluate import evaluate
from pricer.items import Item
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.notebook import tqdm

In [3]:
LITE_MODE = False
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.get_from_hub(dataset)
print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 800,000 training items, 10,000 validation items, 10,000 test items


In [4]:
pipeline = Pipeline(steps = [
    (
        "tfidf", 
         TfidfVectorizer(
                    max_features=2000,
                    stop_words="english",
                    ngram_range=(1, 2),
                    sublinear_tf=True
                )
    ),
    (
        "model", 
        SVR()
    )
])

In [5]:
def get_dataframe(train, test):
    train_df = [{"text": item.summary, "price": float(item.price)} for item in tqdm(train)]
    test_df = [{"text": item.summary, "price": float(item.price)} for item in tqdm(test)]
    return pd.DataFrame(train_df), pd.DataFrame(test_df)

In [6]:
train_df , test_df  = get_dataframe(train , test)

  0%|          | 0/800000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
pipeline.fit(train_df['text'], train_df['price'])

In [ ]:
pipeline.predict([test[0].summary])[0]

In [9]:
def predict_svr(item):
    return pipeline.predict([item.summary])[0]

In [ ]:
evaluate(predict_svr, test)

### SVR didnt perform really well and its just predicting almost the mean